In [9]:
import os
import sys
import numpy as np 
import threading
import time
import signal
import pandas as pd
import copy

module_path = os.path.abspath("../")
sys.path.append(module_path)
module_path = os.path.abspath("../ift6758/client")
sys.path.append(module_path)

from serving_client import ServingClient
from game_client import GameClient


In [2]:
ip = '127.0.0.1'
port = '30000'
            
workspace = 'ds-team-9'
# model_name -> 'XGBoost', 'best-hist-gradient-boosting', 'logistic-regression-shot-angle', 
#               'logistic-regression-shot-distance', 'logistic-regression-shot-distance-and-angle'
model_name = 'XGBoost'
version = '1.0.0'

features = ['team', 'game_seconds', 'period', 'coord_x', 'coord_y', 'shot_distance']

In [3]:
game_client = GameClient()
serving_client = ServingClient(ip, port, features)

In [4]:
# For testing it is better to run the test.py file on the terminal so that you can stop the thread by "control+c"

exit_event = threading.Event()

def get_live_data(game_id, features):
    res_features = copy.deepcopy(features)
    res_features.append('prediction')
    res_features.append('label')
    predicted_df = pd.DataFrame(columns=res_features)
    df_index = 0
    while not game_client.get_game_status(game_id)[0] == 'FINAL':
        game_df = game_client.get_game_df(game_id)
        if len(game_df) > df_index:
            result_df = game_df.loc[(game_df.index >= df_index)]
            #print(result_df)
            result = serving_client.predict(result_df)
            print(result)
            predicted_df = pd.concat([predicted_df, result], axis=0)
            #print(len(predicted_df))
            df_index = len(predicted_df)
            #print(predicted_df)
        time.sleep(60)
        
        if exit_event.is_set():
            break
            
def signal_handler(signum, frame):
    exit_event.set()

In [5]:
serving_client.download_registry_model(workspace, model_name, version)

{'workspace': 'ds-team-9', 'model_name': 'XGBoost', 'version': '1.0.0'}

In [6]:
today_games = game_client.get_today_games()

games_list = []
for game in today_games['dates'][0]['games']:
    games_list.append(game_client.get_game(game))
    
games_list

[{'Home_Team': 'Florida Panthers',
  'Away_Team': 'New Jersey Devils',
  'Game_ID': 2022020518,
  'Game_Status': ('live',
   {'Home_Team': 'Florida Panthers',
    'Away_Team': 'New Jersey Devils',
    'Game_ID': 2022020518,
    'Game_Status': ('2nd', '00:00')})},
 {'Home_Team': 'Detroit Red Wings',
  'Away_Team': 'Tampa Bay Lightning',
  'Game_ID': 2022020519,
  'Game_Status': ('live',
   {'Home_Team': 'Detroit Red Wings',
    'Away_Team': 'Tampa Bay Lightning',
    'Game_ID': 2022020519,
    'Game_Status': ('2nd', '00:00')})},
 {'Home_Team': 'Colorado Avalanche',
  'Away_Team': 'Montréal Canadiens',
  'Game_ID': 2022020520,
  'Game_Status': ('live',
   {'Home_Team': 'Colorado Avalanche',
    'Away_Team': 'Montréal Canadiens',
    'Game_ID': 2022020520,
    'Game_Status': ('1st', '00:00')})},
 {'Home_Team': 'Chicago Blackhawks',
  'Away_Team': 'Nashville Predators',
  'Game_ID': 2022020521,
  'Game_Status': ('live',
   {'Home_Team': 'Chicago Blackhawks',
    'Away_Team': 'Nashville Pre

In [7]:
#game_id = 2022020485
game_id = 2022020519

In [ ]:
if game_client.get_game_status(game_id)[0] == 'FINAL':
    print(game_client.get_game_status(game_id)[1])
    game_df = game_client.get_game_df(game_id)
    result = serving_client.predict(game_df)
elif game_client.get_game_status(game_id)[0] == 'Preview':
    print("Game not started yet")
elif game_client.get_game_status(game_id)[0] == 'live':
    print(game_client.get_game_status(game_id)[1])
    signal.signal(signal.SIGINT, signal_handler)
    th = threading.Thread(target=get_live_data, args=(game_id,features))
    th.start()
    th.join()

{'Home_Team': 'Detroit Red Wings', 'Away_Team': 'Tampa Bay Lightning', 'Game_ID': 2022020519, 'Game_Status': ('2nd', '00:00')}
Processing games...


100%|████████████████████████████████████████████| 1/1 [00:00<00:00, 377.08it/s]

                   team  game_seconds  period  coord_x  coord_y  \
0   Tampa Bay Lightning      0.400000       1     74.0     24.0   
1   Tampa Bay Lightning      1.483333       1     68.0    -20.0   
2   Tampa Bay Lightning      2.366667       1     32.0     14.0   
3   Tampa Bay Lightning      2.483333       1     74.0     36.0   
4   Tampa Bay Lightning      3.216667       1     55.0     18.0   
5   Tampa Bay Lightning      4.033333       1     75.0    -18.0   
6   Tampa Bay Lightning      4.716667       1     62.0      1.0   
7   Tampa Bay Lightning      4.900000       1     64.0     -9.0   
8   Tampa Bay Lightning      5.016667       1     55.0      3.0   
9   Tampa Bay Lightning      5.100000       1     58.0     -4.0   
10  Tampa Bay Lightning      5.266667       1     39.0    -23.0   
11    Detroit Red Wings      6.400000       1    -66.0    -33.0   
12    Detroit Red Wings      7.783333       1    -68.0    -15.0   
13    Detroit Red Wings      8.083333       1    -74.0    -34.



A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Processing games...


100%|████████████████████████████████████████████| 1/1 [00:00<00:00, 485.68it/s]


In [ ]:
def get_live_data(game_id, features):
    res_features = features
    res_features.append('prediction')
    res_features.append('label')
    predicted_df = pd.DataFrame(columns=res_features)
    df_index = 0
    while not game_client.get_game_status(game_id)[0] == 'FINAL':
        game_df = game_client.get_game_df(game_id)
        if len(game_df) > df_index:
            result_df = game_df.loc[(game_df.index >= df_index)]
            #print(result_df)
            result = serving_client.predict(result_df)
            print(result)
            predicted_df = pd.concat([predicted_df, result], axis=0)
            #print(len(predicted_df))
            df_index = len(predicted_df)
            #print(predicted_df)
        time.sleep(60)


In [ ]:
get_live_data(game_id,features)